In [ ]:
# Cell 1: Mount Google Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

# Install ultralytics if not already installed
!pip install ultralytics

In [ ]:
# Cell 2: Import Necessary Libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from IPython.display import Image as IPImage, display
import cv2
import os
import numpy as np

In [ ]:
# Cell 3: Load Models
# Replace with your actual YOLO model path in Drive
yolo_model_path = '/content/drive/MyDrive/path/to/your/yolo_model.pt'  # e.g., '/content/drive/MyDrive/Tomato_dataset/yolo_model.pt'
yolo_model = YOLO(yolo_model_path)

# Replace with your actual Keras model path in Drive
keras_model_path = '/content/drive/MyDrive/path/to/your/keras_model.h5'  # e.g., '/content/drive/MyDrive/Tomato_dataset/keras_classifier.h5'
keras_model = load_model(keras_model_path)

# Define test image paths (modify as needed)
test_images = ['/content/drive/MyDrive/Tomato_dataset/Camera_ripe/WIN_20250905_13_20_47_Pro.jpg']

# Assuming Keras model input shape (e.g., (224, 224, 3)) and classes
# Adjust these based on your Keras model
input_shape = (224, 224)  # Example: Change to match your model's expected input size
class_names = ['Class0', 'Class1']  # Replace with your actual class names, e.g., ['Ripe', 'Unripe']

In [ ]:
# Cell 4: Inference Function with Drawing Boxes (Your Provided Code)
def test_and_draw_boxes(image_path, output_path, model, conf=0.2, iou=0.4):
    if not os.path.exists(image_path):
        print(f"Error: Image {image_path} not found")
        return
    results = model(image_path, conf=conf, iou=iou, max_det=100)
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = model.names[int(box.cls[0])]
            conf = float(box.conf[0])
            color = (0, 255, 0) if label.startswith('G') else (255, 0, 0)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    print(f"Output saved to {output_path}")
    display(IPImage(output_path))

In [ ]:
# Cell 5: Crop and Run Keras Inference Function
def crop_and_keras_predict(image_path, yolo_model, keras_model, conf=0.2, iou=0.4):
    if not os.path.exists(image_path):
        print(f"Error: Image {image_path} not found")
        return

    # Run YOLO to get detections
    results = yolo_model(image_path, conf=conf, iou=iou, max_det=100)
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    predictions = []
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            yolo_label = yolo_model.names[int(box.cls[0])]
            yolo_conf = float(box.conf[0])

            # Crop the detected region
            crop = img[y1:y2, x1:x2]

            # Preprocess the crop for Keras model (resize, normalize, etc.)
            crop_resized = cv2.resize(crop, input_shape[:2])  # Resize to input shape
            crop_array = img_to_array(crop_resized) / 255.0  # Normalize to [0,1] assuming model expects this
            crop_array = np.expand_dims(crop_array, axis=0)  # Add batch dimension

            # Run Keras inference
            keras_pred = keras_model.predict(crop_array)
            keras_class_idx = np.argmax(keras_pred[0])
            keras_class = class_names[keras_class_idx]
            keras_conf = np.max(keras_pred[0])

            # Store or print prediction
            pred_info = {
                'yolo_label': yolo_label,
                'yolo_conf': yolo_conf,
                'keras_class': keras_class,
                'keras_conf': keras_conf,
                'box': (x1, y1, x2, y2)
            }
            predictions.append(pred_info)

            print(f"Crop at {pred_info['box']}: YOLO - {yolo_label} ({yolo_conf:.2f}), Keras - {keras_class} ({keras_conf:.2f})")

    return predictions

In [ ]:
# Cell 6: Run the Tests on Each Image
for img_path in test_images:
    # Define output path for drawn image
    output_path = '/content/output_' + os.path.basename(img_path)

    # Run YOLO and draw boxes
    print(f"Processing {img_path} with YOLO and drawing boxes...")
    test_and_draw_boxes(img_path, output_path, yolo_model)

    # Crop and run Keras predictions
    print(f"Cropping and running Keras inference on {img_path}...")
    predictions = crop_and_keras_predict(img_path, yolo_model, keras_model)

    # Optionally, you can further process predictions, e.g., draw Keras labels on the image too
    # For now, they are printed above